Source for the network creation:

https://www.youtube.com/watch?v=tepxdcepTbY

In this notebook we used yahoo finance api to get data

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
import seaborn as sns

In [ ]:
# getting data from yahoo finance:
! pip install yfinance
import yfinance as yf
df = yf.download("BTC-USD SPY NDX GC=F CL=F DX-Y.NYB", start="2014-09-17",
                    end="2022-09-20" , group_by='tickers', interval = "1d")


df.reset_index(inplace = True)
df['Date'] = pd.to_datetime(df['Date']).dt.date

# we use Forward Fill method to fill the weekend missing data for S&P500:

for culumn in df.columns: 
  df[culumn].fillna(method = 'ffill', inplace = True)


df['Date'] = pd.to_datetime(df['Date'])
# we drop the dates with null bitcoin data
#df.dropna(inplace = True)

df.reset_index(inplace = True, drop=True)
df

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.2 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


[*********************100%***********************]  6 of 6 completed


Date           NDX                                            \
                         Open          High           Low         Close   
0    2014-09-17   4066.760010   4090.949951   4048.580078   4073.570068   
1    2014-09-18   4084.510010   4103.419922   4080.820068   4103.080078   
2    2014-09-19   4113.020020   4118.910156   4084.449951   4100.089844   
3    2014-09-20   4113.020020   4118.910156   4084.449951   4100.089844   
4    2014-09-21   4113.020020   4118.910156   4084.449951   4100.089844   
...         ...           ...           ...           ...           ...   
2920 2022-09-15  12034.429688  12153.339844  11867.879883  11927.490234   
2921 2022-09-16  11775.940430  11875.089844  11710.259766  11861.379883   
2922 2022-09-17  11775.940430  11875.089844  11710.259766  11861.379883   
2923 2022-09-18  11775.940430  11875.089844  11710.259766  11861.379883   
2924 2022-09-19  11753.599609  11956.790039  11752.429688  11953.269531   

                                         GC=F                            ...  \
         Adj Close        Volume         Open         High          Low  ...   
0      4073.570068  1.796710e+09  1236.199951  1236.199951  1221.900024  ...   
1      4103.080078  1.774840e+09  1216.199951  1225.699951  1216.199951  ...   
2      4100.089844  3.178490e+09  1218.300049  1218.300049  1215.300049  ...   
3      4100.089844  3.178490e+09  1218.300049  1218.300049  1215.300049  ...   
4      4100.089844  3.178490e+09  1218.300049  1218.300049  1215.300049  ...   
...            ...           ...          ...          ...          ...  ...   
2920  11927.490234  4.805910e+09  1685.500000  1686.000000  1662.300049  ...   
2921  11861.379883  7.451840e+09  1660.400024  1674.900024  1651.699951  ...   
2922  11861.379883  7.451840e+09  1660.400024  1674.900024  1651.699951  ...   
2923  11861.379883  7.451840e+09  1660.400024  1674.900024  1651.699951  ...   
2924  11953.269531  4.168670e+09  1661.800049  1669.400024  1658.500000  ...   

        DX-Y.NYB                                        SPY              \
             Low       Close   Adj Close Volume        Open        High   
0      83.940002   84.699997   84.699997    0.0  200.770004  201.679993   
1      84.220001   84.320000   84.320000    0.0  201.360001  201.850006   
2      84.230003   84.800003   84.800003    0.0  201.520004  201.899994   
3      84.230003   84.800003   84.800003    0.0  201.520004  201.899994   
4      84.230003   84.800003   84.800003    0.0  201.520004  201.899994   
...          ...         ...         ...    ...         ...         ...   
2920  109.419998  109.739998  109.739998    0.0  392.959991  395.959991   
2921  109.480003  109.760002  109.760002    0.0  384.140015  386.250000   
2922  109.480003  109.760002  109.760002    0.0  384.140015  386.250000   
2923  109.480003  109.760002  109.760002    0.0  384.140015  386.250000   
2924  109.489998  109.739998  109.739998    0.0  382.260010  388.549988   

                                                       
             Low       Close   Adj Close       Volume  
0     199.750000  200.750000  172.454697  151266000.0  
1     201.100006  201.820007  173.373840   94990000.0  
2     200.289993  200.699997  173.217636  121649000.0  
3     200.289993  200.699997  173.217636  121649000.0  
4     200.289993  200.699997  173.217636  121649000.0  
...          ...         ...         ...          ...  
2920  388.779999  390.119995  388.524017   87633800.0  
2921  382.109985  385.559998  385.559998  103084800.0  
2922  382.109985  385.559998  385.559998  103084800.0  
2923  382.109985  385.559998  385.559998  103084800.0  
2924  382.179993  388.549988  388.549988   73278500.0  

[2925 rows x 37 columns]

In [ ]:
# converting column names:

# getting column names:
new_columns = []
for culumn in df.columns:
  new_element =' '.join(culumn)
  new_columns.append(new_element)

# changing the names:
df.columns = new_columns


In [ ]:
# rerodering the columns:

df = df[['Date ', 'BTC-USD Close','CL=F Open', 'CL=F High', 'CL=F Low', 'CL=F Close',
       'CL=F Adj Close', 'CL=F Volume', 'GC=F Open', 'GC=F High', 'GC=F Low',
       'GC=F Close', 'GC=F Adj Close', 'GC=F Volume', 'DX-Y.NYB Open',
       'DX-Y.NYB High', 'DX-Y.NYB Low', 'DX-Y.NYB Close', 'DX-Y.NYB Adj Close',
       'DX-Y.NYB Volume', 'BTC-USD Open', 'BTC-USD High', 'BTC-USD Low',
        'BTC-USD Adj Close', 'BTC-USD Volume', 'NDX Open',
       'NDX High', 'NDX Low', 'NDX Close', 'NDX Adj Close', 'NDX Volume',
       'SPY Open', 'SPY High', 'SPY Low', 'SPY Close', 'SPY Adj Close',
       'SPY Volume']]

df

,Date,BTC-USD Close,CL=F Open,CL=F High,CL=F Low,CL=F Close,CL=F Adj Close,CL=F Volume,GC=F Open,GC=F High,...,NDX Low,NDX Close,NDX Adj Close,NDX Volume,SPY Open,SPY High,SPY Low,SPY Close,SPY Adj Close,SPY Volume
0,2014-09-17,457.334015,94.709999,95.059998,93.739998,94.419998,94.419998,305832.0,1236.199951,1236.199951,...,4048.580078,4073.570068,4073.570068,1.796710e+09,200.770004,201.679993,199.750000,200.750000,172.454697,151266000.0
1,2014-09-18,424.440002,93.970001,94.820000,92.849998,93.070000,93.070000,170772.0,1216.199951,1225.699951,...,4080.820068,4103.080078,4103.080078,1.774840e+09,201.360001,201.850006,201.100006,201.820007,173.373840,94990000.0
2,2014-09-19,394.795990,92.970001,93.220001,91.849998,92.410004,92.410004,92780.0,1218.300049,1218.300049,...,4084.449951,4100.089844,4100.089844,3.178490e+09,201.520004,201.899994,200.289993,200.699997,173.217636,121649000.0
3,2014-09-20,408.903992,92.970001,93.220001,91.849998,92.410004,92.410004,92780.0,1218.300049,1218.300049,...,4084.449951,4100.089844,4100.089844,3.178490e+09,201.520004,201.899994,200.289993,200.699997,173.217636,121649000.0
4,2014-09-21,398.821014,92.970001,93.220001,91.849998,92.410004,92.410004,92780.0,1218.300049,1218.300049,...,4084.449951,4100.089844,4100.089844,3.178490e+09,201.520004,201.899994,200.289993,200.699997,173.217636,121649000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2920,2022-09-15,19701.210938,88.910004,89.150002,84.529999,85.099998,85.099998,235358.0,1685.500000,1686.000000,...,11867.879883,11927.490234,11927.490234,4.805910e+09,392.959991,395.959991,388.779999,390.119995,388.524017,87633800.0
2921,2022-09-16,19772.583984,85.160004,86.589996,84.269997,85.110001,85.110001,87829.0,1660.400024,1674.900024,...,11710.259766,11861.379883,11861.379883,7.451840e+09,384.140015,386.250000,382.109985,385.559998,385.559998,103084800.0
2922,2022-09-17,20127.576172,85.160004,86.589996,84.269997,85.110001,85.110001,87829.0,1660.400024,1674.900024,...,11710.259766,11861.379883,11861.379883,7.451840e+09,384.140015,386.250000,382.109985,385.559998,385.559998,103084800.0
2923,2022-09-18,19419.505859,85.160004,86.589996,84.269997,85.110001,85.110001,87829.0,1660.400024,1674.900024,...,11710.259766,11861.379883,11861.379883,7.451840e+09,384.140015,386.250000,382.109985,385.559998,385.559998,103084800.0


In [ ]:
#Separate dates for future plotting
train_dates = pd.to_datetime(df['Date '])
print(train_dates.tail(15)) #Check last few dates. 


2910   2022-09-05
2911   2022-09-06
2912   2022-09-07
2913   2022-09-08
2914   2022-09-09
2915   2022-09-10
2916   2022-09-11
2917   2022-09-12
2918   2022-09-13
2919   2022-09-14
2920   2022-09-15
2921   2022-09-16
2922   2022-09-17
2923   2022-09-18
2924   2022-09-19
Name: Date , dtype: datetime64[ns]


In [ ]:
#Variables for training
cols = list(df)[1:]
#Date and volume columns are not used in training.     ###### I changed it and used volume.
print(cols) #['Open', 'High', 'Low', 'Close', 'Volume]

['BTC-USD Close', 'CL=F Open', 'CL=F High', 'CL=F Low', 'CL=F Close', 'CL=F Adj Close', 'CL=F Volume', 'GC=F Open', 'GC=F High', 'GC=F Low', 'GC=F Close', 'GC=F Adj Close', 'GC=F Volume', 'DX-Y.NYB Open', 'DX-Y.NYB High', 'DX-Y.NYB Low', 'DX-Y.NYB Close', 'DX-Y.NYB Adj Close', 'DX-Y.NYB Volume', 'BTC-USD Open', 'BTC-USD High', 'BTC-USD Low', 'BTC-USD Adj Close', 'BTC-USD Volume', 'NDX Open', 'NDX High', 'NDX Low', 'NDX Close', 'NDX Adj Close', 'NDX Volume', 'SPY Open', 'SPY High', 'SPY Low', 'SPY Close', 'SPY Adj Close', 'SPY Volume']


In [ ]:
#New dataframe with only training data - 5 columns
df_for_training = df[cols].astype(float)

In [ ]:
df_for_training.shape

(2925, 36)

In [ ]:
#LSTM uses sigmoid and tanh that are sensitive to magnitude so values need to be normalized
# normalize the dataset
scaler = StandardScaler()
scaler = scaler.fit(df_for_training)
df_for_training_scaled = scaler.transform(df_for_training)

In [ ]:
df_for_training_scaled

array([[-0.74354539,  2.0291388 ,  1.96064702, ..., -1.11579833,
        -1.16195731,  1.09305569],
       [-0.74554731,  1.98807076,  1.9475396 , ..., -1.10259249,
        -1.15148661, -0.01981167],
       [-0.74735144,  1.93257325,  1.86015613, ..., -1.11641546,
        -1.15326606,  0.50737453],
       ...,
       [ 0.45358336,  1.49913783,  1.49806027, ...,  1.16509373,
         1.2656939 ,  0.14026436],
       [ 0.41049028,  1.49913783,  1.49806027, ...,  1.16509373,
         1.2656939 ,  0.14026436],
       [ 0.41807482,  1.50191245,  1.47785309, ...,  1.20199566,
         1.29975524, -0.44916023]])

In [ ]:
#As required for LSTM networks, we require to reshape an input data into n_samples x timesteps x n_features. 
#In this example, the n_features is 5. We will make timesteps = 14 (past days data used for training). 

#Empty lists to be populated using formatted training data
trainX = []
trainY = []

#n_future = 1   # Number of days we want to look into the future based on the past days.
n_future = 1

#n_past = 16  # Number of past days we want to use to predict the future.
n_past = 30

In [ ]:
#Reformat input data into a shape: (n_samples x timesteps x n_features)
#In my example, my df_for_training_scaled has a shape (12823, 5)
#12823 refers to the number of data points and 5 refers to the columns (multi-variables).
for i in range(n_past, len(df_for_training_scaled) - n_future +1):
    trainX.append(df_for_training_scaled[i - n_past:i, 0:df_for_training.shape[1]])
    trainY.append(df_for_training_scaled[i + n_future - 1:i + n_future, 0])

In [ ]:
print(trainY)

[array([-0.74802321]), array([-0.74755557]), array([-0.74767096]), array([-0.74807878]), array([-0.74785786]), array([-0.74805973]), array([-0.74956546]), array([-0.74956985]), array([-0.75024381]), array([-0.74979144]), array([-0.74989581]), array([-0.74961409]), array([-0.75095465]), array([-0.75036346]), array([-0.7507885]), array([-0.75155364]), array([-0.75154493]), array([-0.75144378]), array([-0.75126498]), array([-0.7507176]), array([-0.75012093]), array([-0.75053934]), array([-0.75035232]), array([-0.74927048]), array([-0.74904773]), array([-0.74900081]), array([-0.74560081]), array([-0.7457728]), array([-0.74716752]), array([-0.74848727]), array([-0.74777223]), array([-0.74780108]), array([-0.74854417]), array([-0.74821815]), array([-0.74960058]), array([-0.75002611]), array([-0.74990001]), array([-0.74900823]), array([-0.74844053]), array([-0.74853505]), array([-0.74895973]), array([-0.74888061]), array([-0.74846816]), array([-0.74852634]), array([-0.74837078]), array([-0.74

In [ ]:
trainX, trainY = np.array(trainX), np.array(trainY)


In [ ]:
print('trainX shape == {}.'.format(trainX.shape))
print('trainY shape == {}.'.format(trainY.shape))

trainX shape == (2895, 30, 36).
trainY shape == (2895, 1).


In [ ]:


model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(trainX.shape[1], trainX.shape[2]), return_sequences=True))
model.add(LSTM(32, activation='relu', return_sequences=False))

model.add(Dropout(0.4))

model.add(Dense(trainY.shape[1]))

model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            25856     
                                                                 
 lstm_1 (LSTM)               (None, 32)                12416     
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 38,305
Trainable params: 38,305
Non-trainable params: 0
_________________________________________________________________


In [ ]:
np.argwhere(np.isnan(trainX))

array([], shape=(0, 3), dtype=int64)

In [ ]:
from pandas.core.common import random_state
# fit the model
history = model.fit(trainX, trainY, epochs=10, batch_size=16, validation_split=0.1, verbose=1)

plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.legend()

In [ ]:
#Remember that we can only predict one day in future as our model needs 5 variables
#as inputs for prediction. We only have all 5 variables until the last day in our dataset.
#n_past = 15
n_past = 15

#n_days_for_prediction=16  #let us predict past 15 days

n_days_for_prediction=16

In [ ]:
predict_period_dates = pd.date_range(list(train_dates)[-n_past], periods=n_days_for_prediction).tolist()
print(predict_period_dates)

[Timestamp('2022-09-11 00:00:00', freq='D'), Timestamp('2022-09-12 00:00:00', freq='D'), Timestamp('2022-09-13 00:00:00', freq='D'), Timestamp('2022-09-14 00:00:00', freq='D'), Timestamp('2022-09-15 00:00:00', freq='D'), Timestamp('2022-09-16 00:00:00', freq='D'), Timestamp('2022-09-17 00:00:00', freq='D'), Timestamp('2022-09-18 00:00:00', freq='D'), Timestamp('2022-09-19 00:00:00', freq='D'), Timestamp('2022-09-20 00:00:00', freq='D'), Timestamp('2022-09-21 00:00:00', freq='D'), Timestamp('2022-09-22 00:00:00', freq='D'), Timestamp('2022-09-23 00:00:00', freq='D'), Timestamp('2022-09-24 00:00:00', freq='D'), Timestamp('2022-09-25 00:00:00', freq='D'), Timestamp('2022-09-26 00:00:00', freq='D')]


In [ ]:
prediction = model.predict(trainX[-n_days_for_prediction:]) #shape = (n, 1) where n is the n_days_for_prediction


1/1 [==============================] - 0s 248ms/step


In [ ]:
#Perform inverse transformation to rescale back to original range
#Since we used 5 variables for transform, the inverse expects same dimensions
#Therefore, let us copy our values 5 times and discard them after inverse transform
prediction_copies = np.repeat(prediction, df_for_training.shape[1], axis=-1)
y_pred_future = scaler.inverse_transform(prediction_copies)[:,0]

In [ ]:
# Convert timestamp to date
forecast_dates = []
for time_i in predict_period_dates:
    forecast_dates.append(time_i.date())

In [ ]:
df_forecast = pd.DataFrame({'Date':np.array(forecast_dates), 'btc_close':y_pred_future})
df_forecast['Date']=pd.to_datetime(df_forecast['Date'])

In [ ]:
original = df[['Date ', 'BTC-USD Close']]
original['Date ']=pd.to_datetime(original['Date '])
original = original.loc[original['Date '] >= '2020-5-1']

sns.lineplot(original['Date '], original['BTC-USD Close'])
sns.lineplot(df_forecast['Date'], df_forecast['btc_close'])

In [ ]:
df.tail(30)

In [ ]:
df_forecast
